# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Built-in LangChain tools

In [ ]:
#!pip install -U wikipedia

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

**We initialize the LLM**
- it's important here to set the *temperature* to 0 (degree of randmoness) as we're using the LLM as the reasoning engine for the agent to connect to other sources of data

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

**We are loading 2 tools here: *math* and *wikipedia***
- *math* is a Chain itself, which uses an LLM in conjunction with a calculator

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

**We initialize the agent**
- tools (loaded above)
- LLM (declared before)
- type of agent, here **CHAT_ZERO_SHOT_REACT_DESCRIPTION**
    - an agent optimized to work with Chatmodels as we use ChatOpenAI in this example *(use different one for Llama?)*
    - *React* is a prompting technique to get the best reasoning perfomance out of the LLM
- we also set *handle_parsing_errors* to *True*
    - if the LLM outputs misformatted responses which cannot be turned into actions, we pass them back into the LLM to correct itself

In [ ]:
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Prompt the agent with a task**
- it uses the *math* Chain to come up with a response
- try different prompts, see the chain of thought (if *verbose* and *debug* are *True*)

In [ ]:
agent("What is the 25% of 300?")

**This is a more interesting example**

In [ ]:
agent("When I was 6, my sister was 3 years younger than me. Now I am 65. How old is my sister?")  # it worked the first time for me

## Wikipedia example
- we ask a question, the agent realizes it should use Wikipedia to find a response
- it will find two different Wikipedia entries for the name we specified and uses its chain of thought to go for the more plausible one
- if *debug* is *True* we can see the chain of thought again (with different colors for different steps)

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 

## Python Agent
- this is probably the most exciting one, similar to *ChatGPT* with *CodeInterpreter*
- we use the same LLM as before and give it the **PythonREPLTool** tool
    - an environment for executing code, similar to Jupyter Notebook 

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

**The results of our prompt are passed back into the agent, so it can decide what to do next**
- this is a pretty straightforward task and works

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

- this instead is a bit harder, and the LLM failed to do it for me
- it's important to **use the *print* statement** to feed the output back into the LLM

In [ ]:
def test_function(num1, num2):
    num3 = (num2 + num1) 
    num3 = num3 / num2
    return num3

num1 = 1
num2 = 5
    
agent.run(f"""
Run this function with these \
two variables and print the output: \
{test_function}, {num1}, {num2}""") 

#### View detailed outputs of the chains

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

## Define your own tool
- you can connect LangChain to your own custom tool/source of data

In [ ]:
#!pip install DateTime

**First we import the *tool* decorator**
- can be applied to any function to turn it into a tool

from langchain.agents import tool
from datetime import date

**In addition to the function itself, we also add a very detailed docstring**
- the docstring will help the agent to know when and how to call this function

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

**We create another agent**
- this time we add the time tool to the tools used before (for the previous agent)

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

### Source: https://learn.deeplearning.ai/langchain/lesson/7/agents